# Using LangChain to handle Professor queries about students

### 0. Prerequisites/Comments

<br /> 
<font size="3">
    
- This notebook makes use of a custom api who code to get a SQL query results. The code for this api has been made available in the attached notebook titled Flask_APi. It is assumed that all the cells in the  Flask API notebook have been and the api is up and running at: http://127.0.0.1:5000
<br/>            
- This notebook makes use of an openai model. It is assumed that an open api key is available in order to run step 5 in this notebook.
<br/>

- The code in this notebook has been tested using a Python 3 kernel.
<br/>
- All the installers are mentioned in step 1. No other installation is required.
<br/>
- Code makes use of SQLite database instead of Postgres as it does not need any installation to work  and demonstrates all the Langchain capabilities well. In production settings, this code can be easily adapted to use Postgres.
<br/>
- The student.db file (in the same folder) is the sqlite db file that has been created with example data. It contains two tables: students, subject_scores. 
This student.db file can be kept in the same folder as the python notebook to work. For reference, all the SQL statements to create the student.db have also been included in the notebook as commented out lines. Code to add rows to the table are included in the notebook 'LangChain' as commented out lines.
    
<br /> 
</font>

## 1a. Running Instructions

<br /> 
<font size="3">
1. All the cells can be run individually, or for similicity, from the menu, choose 'Kernel-> Restart  & Run All'. Step 4 will require a manual input of openapi key.
</font>

## 1b. Testing Instructions
<br /> 
<font size="3">
1. A set of questions have been provided at the end of the notebook in section 11, which can be used to invoke the LangChain developed in this code. More similar questions can be created to test the chain.

</font>

## 2. Supported Features

<br /> 
<font size="3">
    
1. Used LangChain to understand professor's question's intent from text question.

<br />
    
2. Added support for following question types:
    - "Add a new student with a given name and id"
    - "Add a score of x for a student with name Y for a subject Z"
    - "What subjects a student with name Y take?"
    - "Summarize all student scores in a specific subject. Display mean and median scores."
    - "Rank students based on their scores in a particular subject"

<br />
    
3. Based on the extracted intent, make API calls to an API created in notebook 'Flask_api' which queries the SQLite database and returns response.
    
<br />
    
4. Convert the SQL response from Flask api into natural language response.

<br />
    
5. Automate the process using a single SQL query chain that can be invoked with a question in a one line code.
</font>

## 3. Install the required packages

In [1]:
pip install --upgrade --quiet  langchain langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


## 4. Import the required libraries

In [2]:
#Library for querying the Flask api
import requests

#Library to access the open api's gpt-3.5-turbo LLM as a service
import openai

#Misc Libraries
import os
import json
from getpass import getpass

#Library to access the sqlite3 database
import sqlite3

#Langchain libraries
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

## 5. Connect to openai

#### Note: An openai api key will be needed for this step

In [3]:
openapi_key = getpass()
os.environ['OPENAI_API_KEY'] = openapi_key

········


## 6. Test the SQLite Database and tables

<br /> 
<font size="3">
    
Note:
- SQLite database has been used in this assignment instead of Postgres as it does not need any installation for this notebook to work and demonstrates all the Langchain capabilities well.

<br />
    
- In a production setting, Postgres database can be easily adapted to be used in this notebook by updating the db variable. Rest all the python code will remain the same.
    
</font>

In [4]:
# Specify the database to connect to
db = SQLDatabase.from_uri("sqlite:///student.db")

print(db.dialect)  # Database type
print(db.get_usable_table_names())  # List of usable table names

# Execute a test SQL statement from the students table
db.run("SELECT * FROM students LIMIT 10;")

sqlite
['students', 'subject_scores']


"[(1, 'John', 'Smith'), (2, 'Jane', 'Doe'), (3, 'Adam', 'Krebs'), (4, 'Julia', 'Chen'), (5, 'Mike', 'Wang'), (6, 'Justin', 'Arnold'), (7, 'Kate', 'Schlif'), (8, 'June', 'Al'), (9, 'Kim', 'Roland')]"

## 7. Create a SQL query chain using Langchain

<br /> 
<font size="3">
    
### Note:

- gpt-3.5-turbo model from open ai is being used to understand the intent of the input queries.
    
- A temperature setting of 0 is used because we want to the model to have low creativity and have predictable ouput for the same queries.
    
- A sql query chain is created using the llm as well as the sqlite database.
    
</font>

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)

## 8. Manually test the chain to understand intent from text and generate queries

<br /> 
<font size="3">
    
#### Note: 

- The chain from step 5 can be invoked to generate a sql statement which can be then used to get a response from a Flask api manually.
    
- Different types of queries (some commented below) are used to test the system manually
    
</font>

## 8a. Invoke chain to generate a sql query

<br /> 
<font size="3">
    
#### Note:

- Below commented queries were used to create records in the tables and do some basic querying. They are made available here for testing purposes.
- Uncomment on an as need basis
- A query to check total number of students has been uncommented for demp purpose.
- The chain outputs a generated query based on the input question.
  
</font>


In [6]:
#Add new Students
#sudent.db contains this data already. Uncomment only if creating a new db
#response = chain.invoke({"question": "Add a new student named John Smith with ID 1. Limit to only one query. Add None for empty columns"})
#response = chain.invoke({"question": "Add a new student named Jane Doe with ID 2. Limit to only one query. Add None for empty columns"})
#response = chain.invoke({"question": "Add a new student named Adam Krebs with ID 3. Limit to only one query. Add None for empty columns"})
#response = chain.invoke({"question": "Add a new student named Julia Chen with ID 4. Limit to only one query. Add None for empty columns"})
#response = chain.invoke({"question": "Add a new student named Mike Wang with ID 5. Limit to only one query. Add None for empty columns"})
#response = chain.invoke({"question": "Add a new student named Justin Arnold with ID 6. Limit to only one query. Add None for empty columns"})


#Add new scores
#sudent.db contains this data already. Uncomment only if creating a new db
#response = chain.invoke({"question": "Add a score of 90 for John Smith in Math. Autogenerate id as one more than the largest id"})
#response = chain.invoke({"question": "Add a score of 81 for John Smith in Physics. Autogenerate id as one more than the largest id"})
#response = chain.invoke({"question": "Add a new score of 84 for Jane Doe in Physics. Autogenerate id as one more than the largest id."})
#response = chain.invoke({"question": "Add a new score of 94 for Adam Krebs in Math. Autogenerate id as one more than the largest id."})
#response = chain.invoke({"question": "Add a new score of 84 for Julia Chen in Chemistry. Autogenerate id as one more than the largest id."})
#response = chain.invoke({"question": "Add a new score of 73 for Mike Wang in Chemistry. Autogenerate id as one more than the largest id."})
#response = chain.invoke({"question": "Add a new score of 83 for Justin Arnold in Physics. Autogenerate id as one more than the largest id."})

#Check questions about which subjects a specific student took
#response = chain.invoke({"question": "What subjects did John Smith take?"})


#Check questions about summarizing scores
#response = chain.invoke({"question": "Summarize all student scores in Physics. Show mean and median scores."})


#Check total number of students
gen_query = chain.invoke({"question": "How many students are there?"})

print(gen_query)

SELECT COUNT(id) AS total_students FROM students;


## 8b. Manually call the Flask based api to get results of the query generated in 6a.

<br /> 
<font size="3">
    
#### Note 

- The api must be running at port 5000 using the attached notebook DwelFi_Flask_app.
- The api is queried using a POST method.
- An input json containing the generated query is passed to the api call.
- The api returns the query output as json content.
    
</font>

In [7]:
def query_api(query):
    data={"qstring":query}
    json_data = json.dumps(data)
    r = requests.post(
        "http://127.0.0.1:5000/api/student/query",
        data=json_data,
        headers={"Content-Type": "application/json"}
        
    )
    #print("Generated Query is", query)
    #print("Query execution result is", r.text)
    return r.text

In [8]:
#Get query result from api
query_result = query_api(gen_query)
print(query_result)

[
    {
        "total_students": 9
    }
]


In [9]:
#Verify that the query result from api matches with results from directly querying the database
verify_query_result = db.run(gen_query)
print(verify_query_result)

[(9,)]


## 9. Automatic Execution of generated SQL queries

<br /> 
<font size="3">
    
#### Note:

- Set up a chain with three automated components:
  * Generate query (write_query)
  * Use query to get response from api call (execute_query)
  * Formate the api call's ouput in proper format to display output (parser)
    
</font>

In [10]:
#Create an automated chain
execute_query = query_api
write_query = create_sql_query_chain(llm, db)
parser = JsonOutputParser()
chain = write_query | execute_query | parser
chain.invoke({"question": "How many students are there?"})

[{'total_students': 9}]

## 10. Answering back in Natural Language

<br /> 
<font size="3">
    
#### Note:

- Set up a prompt and a chain to feed query output to  an llm to generate response in natural language.
- Further, create one chain containing all the components to automate the whole process.
    
</font>

In [11]:
answer_prompt = PromptTemplate.from_template(
    """Based on the user's question, the corresponding SQL query, and the SQL result, answer the user's question.

Question: {question}
SQLQuery: {query}
SQLResult: {result}
Answer: """
)

# Chain to generate answers using the above prompt
answer = answer_prompt | llm | StrOutputParser()

chain = (
    {"query": write_query, "result": write_query | execute_query, "question": RunnablePassthrough()} 
    | answer
)


## 11. Questions to test the constructed chain for answering natural language questions

## Question to check no. of students in the course

In [12]:
chain.invoke({"question": "How many students are there?"})

'There are 9 students in the database.'

## Question to check the subjects taken by a specific student

In [13]:
chain.invoke({"question": "What subjects have been taken by John Smith?"})

'John Smith has taken Math and Physics subjects.'

## Question to summarize all student scores in a particular subject

In [14]:
chain.invoke({"question": "Summarize all student scores in Physics. Show mean and median scores."})

'The mean score of all student scores in Physics is 83.0, and the median score is 84.0.'

## Question to add a new student to the database

In [19]:
chain.invoke({"question": "Add a new student named Jack Mumby with ID 10. Limit to only one query. Add None for empty columns"})


'The new student named Jack Mumby with ID 10 has been successfully added to the database.'

## Question to add a new score for a student in a specific subject

In [20]:
chain.invoke({"question": "Add a new score of 93 for Jack Mumby in Math. Autogenerate id as one more than the largest id."})


'The new score of 93 for Jack Mumby in Math has been successfully added to the database. The id for this record was autogenerated as one more than the largest id in the subject_scores table.'

## Questions to rank students based on scores

In [29]:
chain.invoke({"question": "Who are the top students based on scores in Math in descending order. Limit to only one query"})

'The top students based on scores in Math in descending order are:\n1. Adam Krebs with a score of 94.0\n2. Jack Mumby with a score of 93.0\n3. June Al with a score of 92.0\n4. John Smith with a score of 90.0'

## Question containing usupported operation, gracefully provide an error message.

In [18]:
chain.invoke({"question": "Delete score of 95 for Kate Schlif in Physics."})

'The SQL query to delete the score of 95 for Kate Schlif in Physics encountered an internal server error. This could be due to a server overload or an error in the application. It is recommended to try running the query again later or check for any issues in the application that may be causing the error.'